<div style="color:rgb(0,0,255);font-size: 40px;font-weight:700;">
MAIN SETTINGS
</div>

In [1]:
### SETTINGS (VAST.AI ONLY) ###

import os
import re
import time
import subprocess
import shutil
from concurrent.futures import ThreadPoolExecutor, as_completed
from getpass import getpass
from urllib.parse import urlencode

# Notebook is now Vast.ai-only.
ON_KAGGLE = False
ON_COLAB = False
ON_VAST = True

MAX_PARALLEL_DOWNLOADS = max(1, int(os.environ.get("MAX_PARALLEL_DOWNLOADS", "3")))
MIN_VALID_FILE_BYTES = int(os.environ.get("MIN_VALID_FILE_BYTES", "1000000"))

BASE_DIR = os.environ.get("BASE_DIR", "/workspace")
if not os.path.isdir(BASE_DIR):
    raise FileNotFoundError(f"BASE_DIR not found: {BASE_DIR}")
print("Working directory:", BASE_DIR)

# Configuration
FORGE_DIR = os.path.join(BASE_DIR, "stable-diffusion-webui-forge")
MODELS_DIR = os.path.join(FORGE_DIR, "models", "Stable-diffusion")
LORA_DIR = os.path.join(FORGE_DIR, "models", "Lora")
CONTROLNET_DIR = os.path.join(FORGE_DIR, "extensions", "sd-webui-controlnet")
CONTROLNET_MODELS_DIR = os.path.join(CONTROLNET_DIR, "models")
EXTENSIONS_DIR = os.path.join(FORGE_DIR, "extensions")
OUTPUTS_DIR = os.path.join(FORGE_DIR, "outputs")
VOLUME_DIR = os.path.join(BASE_DIR, "volume")
GEN_DIR = os.path.join(BASE_DIR, "gen")
IMAGES_DIR = os.path.join(GEN_DIR, "Images")

for d in [MODELS_DIR, LORA_DIR, CONTROLNET_MODELS_DIR, EXTENSIONS_DIR, OUTPUTS_DIR, VOLUME_DIR, GEN_DIR, IMAGES_DIR]:
    os.makedirs(d, exist_ok=True)


def ensure_python_package(pkg_name: str, import_name: str | None = None):
    module_name = import_name or pkg_name
    try:
        __import__(module_name)
        print(f"Python package already installed: {pkg_name}")
    except Exception:
        print(f"Installing missing Python package: {pkg_name}")
        subprocess.run(["python", "-m", "pip", "install", "-q", pkg_name], check=False)


def ensure_aria2c():
    if shutil.which("aria2c") is not None:
        print("aria2c already available")
        return

    print("aria2c not found -> installing...")
    subprocess.run(["apt", "update", "-qq"], check=False)
    subprocess.run(["apt", "install", "-y", "-qq", "aria2"], check=False)


# Dependencies used by generation cell and optional downloads
ensure_python_package("openpyxl")
ensure_python_package("requests")
ensure_aria2c()


def get_secret(name: str):
    value = os.environ.get(name)
    if value:
        return value.strip(), "env"
    return None, None


CIVITAI_TOKEN, CIVITAI_SRC = get_secret("CIVITAI_TOKEN")
HF_TOKEN, HF_SRC = get_secret("HF_TOKEN")

if not CIVITAI_TOKEN:
    manual_civitai = getpass("Enter CIVITAI_TOKEN (leave blank to skip): ").strip()
    if manual_civitai:
        CIVITAI_TOKEN, CIVITAI_SRC = manual_civitai, "manual_input"

if not HF_TOKEN:
    manual_hf = getpass("Enter HF_TOKEN (leave blank to skip): ").strip()
    if manual_hf:
        HF_TOKEN, HF_SRC = manual_hf, "manual_input"

TOKENS = {}
if CIVITAI_TOKEN:
    TOKENS["CIVITAI"] = CIVITAI_TOKEN
if HF_TOKEN:
    TOKENS["HF_TOKEN"] = HF_TOKEN

print("Token sources:")
print(f"  CIVITAI_TOKEN: {CIVITAI_SRC or 'not found'}")
print(f"  HF_TOKEN: {HF_SRC or 'not found'}")
if not TOKENS:
    print("No tokens provided: token-protected downloads will be skipped.")


aria2c not found → installing...
aria2c installed successfully
Working directory: /kaggle/working
Token sources:
  CIVITAI_TOKEN: kaggle_secrets
  HF_TOKEN: kaggle_secrets


In [2]:
### FORGE INSTALL/BOOTSTRAP (DISABLED FOR VAST-ONLY NOTEBOOK) ###

print("Skipped: this notebook no longer installs Forge or system packages for Kaggle/Colab.")
print("Expected setup: Forge is already provisioned by your Vast.ai template.")


Checking/installing platform dependencies...




W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)




171 packages can be upgraded. Run 'apt list --upgradable' to see them.
git is already the newest version (1:2.34.1-1ubuntu1.15).
The following additional packages will be installed:
  libpython3.10 libpython3.10-dev libpython3.10-minimal libpython3.10-stdlib
  python3-pip-whl python3-pkg-resources python3-setuptools
  python3-setuptools-whl python3-wheel python3.10 python3.10-dev
  python3.10-minimal python3.10-venv
Suggested packages:
  python-setuptools-doc python3.10-doc binfmt-support
The following NEW packages will be installed:
  python3-pip python3-pip-whl python3-setuptools python3-setuptools-whl
  python3-venv python3-wheel python3.10-venv
The following packages will be upgraded:
  libpython3.10 libpython3.10-dev libpython3.10-minimal libpython3.10-stdlib
  python3-pkg-resources python3.10 python3.10-dev python3.10-minimal
8 upgraded, 7 newly installed, 0 to remove and 163 not upgraded.
Need to get 17.2 MB of archives.
After this operation, 12.4 MB of additional disk space wil

Cloning into '/kaggle/working/stable-diffusion-webui-forge'...


Optional bootstrap finished.


In [3]:
### CONTROLNET INSTALL OPTIONS (DISABLED) ###

print("Skipped: ControlNet install/update is handled by provisioning (woload.sh), not by notebook.")


ControlNet extension path: /kaggle/working/stable-diffusion-webui-forge/extensions/sd-webui-controlnet
Install mode: reinstall
Cloning ControlNet repository...


Cloning into '/kaggle/working/stable-diffusion-webui-forge/extensions/sd-webui-controlnet'...


ControlNet repository is ready
ControlNet models directory: /kaggle/working/stable-diffusion-webui-forge/extensions/sd-webui-controlnet/models

=== ControlNet ===
Parallel downloads: 3
[DOWNLOADING] t2i-adapter_xl_openpose 151 MB
[DOWNLOADING] t2i-adapter_xl_canny 148 MB
[DOWNLOADING] t2i-adapter_xl_sketch 148 MB
[DOWNLOADING] t2i-adapter_diffusers_xl_depth_midas 151 MB[OK]   t2i-adapter_xl_sketch 148 MB -> 147.9 MB

[DOWNLOADING] t2i-adapter_diffusers_xl_depth_zoe 151 MB
[OK]   t2i-adapter_xl_canny 148 MB -> 147.9 MB
[DOWNLOADING] t2i-adapter_diffusers_xl_lineart 151 MB
[OK]   t2i-adapter_xl_openpose 151 MB -> 150.7 MB
[OK]   t2i-adapter_diffusers_xl_depth_zoe 151 MB -> 150.7 MB
[OK]   t2i-adapter_diffusers_xl_depth_midas 151 MB -> 150.7 MB
[OK]   t2i-adapter_diffusers_xl_lineart 151 MB -> 150.7 MB
Done: OK=6, FAIL=0

=== Checkpoints ===
Parallel downloads: 1
[DOWNLOADING] WAI ILL V16.0 6,46 GB
[OK]   WAI ILL V16.0 6,46 GB -> 6616.6 MB
Done: OK=1, FAIL=0

=== LoRA ===
Parallel downloa

In [4]:
### RUN WEBUI (VAST: START EXISTING FORGE ONLY) ###

import os
import time
import subprocess
from pathlib import Path

forge_dir = Path(FORGE_DIR)
if not forge_dir.exists():
    raise FileNotFoundError(
        f"FORGE_DIR not found: {forge_dir}. Use a Vast template with Forge preinstalled."
    )

cmd = ["bash", "webui.sh", "-f", "--xformers", "--api", "--port", "17860"]
run_env = os.environ.copy()
run_env["MPLBACKEND"] = "Agg"
print("Running (background):", " ".join(cmd), "in", forge_dir)

pid_path = forge_dir / "webui.pid"
log_path = forge_dir / "webui.log"


def _pid_alive(pid: int) -> bool:
    try:
        os.kill(pid, 0)
        return True
    except Exception:
        return False


def _log_tail(chars: int = 4000) -> str:
    if not log_path.exists():
        return "(webui.log not found yet)"
    return log_path.read_text(encoding="utf-8", errors="ignore")[-chars:]


running_pid = None
if pid_path.exists():
    try:
        old_pid = int(pid_path.read_text(encoding="utf-8").strip())
        if _pid_alive(old_pid):
            running_pid = old_pid
            print(f"WebUI already running with PID={old_pid}. Можно запускать последнюю ячейку.")
        else:
            pid_path.unlink(missing_ok=True)
    except Exception:
        pid_path.unlink(missing_ok=True)

started_proc = None
if running_pid is None:
    with open(log_path, "a", encoding="utf-8") as log_f:
        started_proc = subprocess.Popen(
            cmd,
            cwd=forge_dir,
            env=run_env,
            stdout=log_f,
            stderr=subprocess.STDOUT,
            start_new_session=True,
            text=True,
        )
    pid_path.write_text(str(started_proc.pid), encoding="utf-8")
    running_pid = started_proc.pid
    print(f"WebUI started in background. PID={running_pid}")
    print(f"Logs: {log_path}")

observe_timeout = int(os.environ.get("FORGE_STARTUP_OBSERVE_TIMEOUT", "120"))
observe_interval = float(os.environ.get("FORGE_STARTUP_OBSERVE_INTERVAL", "2"))

for _ in range(max(1, int(observe_timeout / observe_interval))):
    tail = _log_tail()

    if started_proc is not None:
        rc = started_proc.poll()
        if rc is not None:
            raise RuntimeError(
                f"WebUI process exited early with code {rc}.\n"
                f"Check logs at: {log_path}\n"
                f"Last log lines:\n{tail}"
            )

    if "Running on local URL" in tail or "Uvicorn running" in tail:
        print("WebUI appears ready. Можно запускать последнюю ячейку.")
        break

    time.sleep(observe_interval)
else:
    print("WebUI всё ещё запускается в фоне. Запускайте последнюю ячейку: она дождётся API.")
    print(f"Если API не поднимется, проверьте лог: {log_path}")


Patched torch stage (reinstalled numpy 2.x + scikit-image for ABI compatibility).
Patched start stage (re-pin numpy/scikit-image before importing webui).
Patched CLIP install command (fixed flags + preserved indentation).
soft_inpainting.py already disabled or not present.
Running (background): bash webui.sh -f --xformers --api --port 17860 in /kaggle/working/stable-diffusion-webui-forge
MPLBACKEND forced to: Agg
WebUI started in background. PID=1927
Logs: /kaggle/working/stable-diffusion-webui-forge/webui.log
WebUI всё ещё запускается в фоне. Запускайте последнюю ячейку: она дождётся API.
Если API не поднимется, проверьте лог: /kaggle/working/stable-diffusion-webui-forge/webui.log


In [6]:
### API GENERATION FROM XLSX ###

import os
import re
import json
import glob
import time
import shutil
import zipfile
import base64
from datetime import datetime

import requests
from openpyxl import load_workbook

API_BASE = os.environ.get("FORGE_API_BASE", "http://127.0.0.1:17860")
TXT2IMG_URL = f"{API_BASE}/sdapi/v1/txt2img"
OPENAPI_URL = f"{API_BASE}/openapi.json"

PROMPTS_CANDIDATES = [
    os.path.join(GEN_DIR, "prompts.xlsx"),
    os.path.join(GEN_DIR, "prompts.xlxs"),
    os.path.join(GEN_DIR, "Prompts.xlsx"),
    os.path.join(GEN_DIR, "Prompts.xlxs"),
    os.path.join(BASE_DIR, "prompts.xlsx"),
    os.path.join(BASE_DIR, "prompts.xlxs"),
    os.path.join(BASE_DIR, "Prompts.xlsx"),
    os.path.join(BASE_DIR, "Prompts.xlxs"),
    "/workspace/gen/prompts.xlsx",
    "/workspace/gen/prompts.xlxs",
]

if ON_COLAB:
    PROMPTS_CANDIDATES.extend([
        "/content/prompts.xlsx",
        "/content/prompts.xlxs",
        "/content/Prompts.xlsx",
        "/content/Prompts.xlxs",
    ])

if ON_KAGGLE:
    PROMPTS_CANDIDATES.append("/kaggle/input/datasets/sokolenkotimofei/prompts/Prompts.xlsx")
    for pattern in [
        "/kaggle/input/*/*/prompts/prompts.xlsx",
        "/kaggle/input/*/*/prompts/prompts.xlxs",
        "/kaggle/input/*/*/prompts/Prompts.xlsx",
        "/kaggle/input/*/*/prompts/Prompts.xlxs",
        "/kaggle/input/*/prompts/prompts.xlsx",
        "/kaggle/input/*/prompts/prompts.xlxs",
        "/kaggle/input/*/prompts/Prompts.xlsx",
        "/kaggle/input/*/prompts/Prompts.xlxs",
    ]:
        PROMPTS_CANDIDATES.extend(sorted(glob.glob(pattern)))

PROMPTS_CANDIDATES = list(dict.fromkeys(PROMPTS_CANDIDATES))

API_READY_TIMEOUT = int(os.environ.get("FORGE_API_READY_TIMEOUT", "600"))
API_READY_INTERVAL = float(os.environ.get("FORGE_API_READY_INTERVAL", "3"))


def wait_for_api_ready(base_url: str, timeout_s: int = API_READY_TIMEOUT, interval_s: float = API_READY_INTERVAL):
    start = time.time()
    last_error = None
    webui_log = os.path.join(FORGE_DIR, "webui.log")
    webui_pid = os.path.join(FORGE_DIR, "webui.pid")

    def _log_tail(chars: int = 4000):
        if not os.path.exists(webui_log):
            return "(webui.log not found yet)"
        with open(webui_log, "r", encoding="utf-8", errors="ignore") as f:
            return f.read()[-chars:]

    def _is_pid_alive():
        try:
            if not os.path.exists(webui_pid):
                return None
            with open(webui_pid, "r", encoding="utf-8") as f:
                pid = int(f.read().strip())
            os.kill(pid, 0)
            return pid
        except Exception:
            return False

    while (time.time() - start) < timeout_s:
        try:
            r = requests.get(f"{base_url}/sdapi/v1/progress", timeout=10)
            if r.status_code < 500:
                return True
            last_error = f"HTTP {r.status_code}"
        except Exception as e:
            last_error = str(e)

        pid_state = _is_pid_alive()
        if pid_state is False:
            raise RuntimeError(
                "Forge API недоступен и процесс WebUI уже завершился.\n"
                f"Last error: {last_error}\n"
                f"WebUI log tail:\n{_log_tail()}"
            )

        time.sleep(interval_s)

    raise RuntimeError(
        f"Forge API is not reachable at {base_url} after {timeout_s}s. "
        f"Last error: {last_error}.\n"
        f"WebUI log tail:\n{_log_tail()}\n"
        "Run the Forge startup cell first. If this is first launch, increase FORGE_API_READY_TIMEOUT (e.g. 900)."
    )


LOG_PATH = os.path.join(VOLUME_DIR, "log.txt")
API_IMAGES_DIR = os.path.join(OUTPUTS_DIR, "api_generated")
REQUEST_DUMPS_DIR = os.path.join(VOLUME_DIR, "request_dumps")
os.makedirs(API_IMAGES_DIR, exist_ok=True)
os.makedirs(REQUEST_DUMPS_DIR, exist_ok=True)

INPUT_IMAGE_DIRS = [
    os.path.join(GEN_DIR, "Images"),
    os.path.join(GEN_DIR, "images"),
    "/kaggle/input/datasets/sokolenkotimofei/images",
]
if ON_KAGGLE:
    INPUT_IMAGE_DIRS.extend(sorted(glob.glob("/kaggle/input/*/*/images")))
    INPUT_IMAGE_DIRS.extend(sorted(glob.glob("/kaggle/input/*/images")))
INPUT_IMAGE_DIRS = [p for i, p in enumerate(INPUT_IMAGE_DIRS) if p and p not in INPUT_IMAGE_DIRS[:i]]

CONFLICT_RULES = [
    (("hr_resize_x", "hr_resize_y"), ("hr_scale",)),
]

CONTROLNET_FIELDS = [
    "enabled", "image", "mask", "weight", "module", "model", "resize_mode", "lowvram",
    "processor_res", "threshold_a", "threshold_b", "guidance_start", "guidance_end",
    "control_mode", "pixel_perfect",
]


def log_line(text: str):
    ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    line = f"[{ts}] {text}"
    print(line)
    with open(LOG_PATH, "a", encoding="utf-8") as f:
        f.write(line + "\n")


def normalize_value(value):
    if isinstance(value, str):
        stripped = value.strip()
        lowered = stripped.lower()
        if lowered in {"", "null", "none", "nan"}:
            return None
        if lowered in {"true", "yes", "1"}:
            return True
        if lowered in {"false", "no", "0"}:
            return False
        if re.fullmatch(r"-?\d+", stripped):
            return int(stripped)
        if re.fullmatch(r"-?\d+\.\d+", stripped):
            return float(stripped)
        return stripped
    return value


def first_existing_path(candidates):
    for p in candidates:
        if os.path.exists(p):
            return p
    return None


def parse_workbook(path):
    wb = load_workbook(path, data_only=True)
    ws = wb.active

    row1 = [normalize_value(v) for v in next(ws.iter_rows(min_row=1, max_row=1, values_only=True))]
    row2 = [normalize_value(v) for v in next(ws.iter_rows(min_row=2, max_row=2, values_only=True))]

    row1_headers_like = any(isinstance(v, str) and v in {"prompt", "negative_prompt", "sampler_name", "steps", "cn1_enabled"} for v in row1)

    if row1_headers_like:
        instruction = "{prompt}"
        headers = [str(v).strip() if v is not None else "" for v in row1]
        data_start_row = 2
    else:
        instruction_parts = [str(v).strip() for v in row1 if v is not None and str(v).strip()]
        if not instruction_parts:
            raise ValueError("Первая строка (инструкция) пустая")
        instruction = " ".join(instruction_parts)
        headers = [str(v).strip() if v is not None else "" for v in row2]
        data_start_row = 3

    if not any(headers):
        raise ValueError("Не найдены заголовки таблицы с переменными")

    rows = []
    for row_idx in range(data_start_row, ws.max_row + 1):
        row_values = [normalize_value(v) for v in next(ws.iter_rows(min_row=row_idx, max_row=row_idx, values_only=True))]
        if all(v is None for v in row_values):
            continue

        row_map = {}
        for idx, key in enumerate(headers):
            if not key or key.startswith("S_"):
                continue
            value = row_values[idx] if idx < len(row_values) else None
            if value is None:
                continue
            row_map[key] = value
        rows.append(row_map)

    if not rows:
        raise ValueError("Не найдено строк данных")

    return instruction, rows


def render_instruction(template: str, variables: dict):
    def repl(match):
        key = match.group(1)
        return str(variables.get(key, match.group(0)))
    return re.sub(r"\{([a-zA-Z0-9_]+)\}", repl, template)


def apply_conflict_rules(payload: dict):
    cleaned = dict(payload)
    for primary_keys, conflicting_keys in CONFLICT_RULES:
        primary_present = all((k in cleaned and cleaned[k] is not None) for k in primary_keys)
        if primary_present:
            for ck in conflicting_keys:
                cleaned.pop(ck, None)
    return cleaned


def fetch_txt2img_params_dump():
    try:
        response = requests.get(OPENAPI_URL, timeout=30)
        response.raise_for_status()
        spec = response.json()
        schemas = spec.get("components", {}).get("schemas", {})
        candidates = [
            "StableDiffusionTxt2ImgProcessingApi",
            "Txt2ImgRequest",
            "StableDiffusionProcessingTxt2Img",
        ]
        for name in candidates:
            if name in schemas:
                props = schemas[name].get("properties", {})
                return json.dumps({k: v.get("type", "unknown") for k, v in props.items()}, ensure_ascii=False, indent=2)
        return json.dumps(spec.get("paths", {}).get("/sdapi/v1/txt2img", {}), ensure_ascii=False, indent=2)
    except Exception as e:
        return f"Не удалось получить список переменных: {e}"


def resolve_image_path(image_ref):
    if image_ref is None:
        return None
    if isinstance(image_ref, str) and image_ref.startswith("data:image"):
        return image_ref

    ref = str(image_ref).strip()
    if not ref:
        return None

    if os.path.isabs(ref) and os.path.exists(ref):
        return ref

    candidates = [ref]
    if not os.path.splitext(ref)[1]:
        candidates += [f"{ref}.png", f"{ref}.jpg", f"{ref}.jpeg", f"{ref}.webp"]

    for base_dir in INPUT_IMAGE_DIRS:
        if not os.path.isdir(base_dir):
            continue
        for candidate in candidates:
            full = os.path.join(base_dir, candidate)
            if os.path.exists(full):
                return full

    for pattern in [f"**/{ref}", f"**/{ref}.*"]:
        for base_dir in INPUT_IMAGE_DIRS:
            if not os.path.isdir(base_dir):
                continue
            matches = glob.glob(os.path.join(base_dir, pattern), recursive=True)
            if matches:
                return matches[0]

    return None


def image_to_base64(image_ref):
    if image_ref is None:
        return None
    if isinstance(image_ref, str) and image_ref.startswith("data:image"):
        return image_ref

    image_path = resolve_image_path(image_ref)
    if not image_path:
        raise FileNotFoundError(f"Image/Mask not found: {image_ref}. Search dirs: {INPUT_IMAGE_DIRS}")

    ext = os.path.splitext(image_path)[1].lower()
    mime = {
        ".png": "image/png",
        ".jpg": "image/jpeg",
        ".jpeg": "image/jpeg",
        ".webp": "image/webp",
        ".bmp": "image/bmp",
    }.get(ext, "application/octet-stream")

    with open(image_path, "rb") as f:
        b64 = base64.b64encode(f.read()).decode("utf-8")
    return f"data:{mime};base64,{b64}"


def parse_override_settings(value):
    if value in (None, "", {}):
        return {}
    if isinstance(value, dict):
        return value
    if isinstance(value, str):
        txt = value.strip()
        if not txt:
            return {}
        if txt.startswith("{"):
            return json.loads(txt)
        return json.loads("{" + txt + "}")
    raise ValueError(f"Unsupported override_settings format: {type(value)}")

def deep_clean(value):
    if isinstance(value, dict):
        cleaned = {}
        for k, v in value.items():
            cv = deep_clean(v)
            if cv is None:
                continue
            if isinstance(cv, str) and not cv.strip():
                continue
            if isinstance(cv, dict) and not cv:
                continue
            cleaned[k] = cv
        return cleaned

    if isinstance(value, list):
        cleaned_list = []
        for item in value:
            ci = deep_clean(item)
            if ci is None:
                continue
            if isinstance(ci, str) and not ci.strip():
                continue
            if isinstance(ci, dict) and not ci:
                continue
            cleaned_list.append(ci)
        return cleaned_list

    if isinstance(value, str):
        stripped = value.strip()
        if stripped == "[]":
            return []
        if stripped.lower() in {"", "null", "none", "nan"}:
            return None
        return stripped

    return value


def dump_payload(payload, generation_idx):
    dump_path = os.path.join(REQUEST_DUMPS_DIR, f"request_{generation_idx:04d}.json")
    with open(dump_path, "w", encoding="utf-8") as f:
        json.dump(payload, f, ensure_ascii=False, indent=2)
    return dump_path


def build_payload(row_values, instruction_template):
    payload = dict(row_values)

    if "prompt" not in payload:
        payload["prompt"] = render_instruction(instruction_template, row_values)

    payload = {k: v for k, v in payload.items() if v is not None}

    if "override_settings" in payload:
        payload["override_settings"] = parse_override_settings(payload.get("override_settings"))

    controlnet_args = []
    for idx in (1, 2, 3):
        unit = {}
        for field in CONTROLNET_FIELDS:
            key = f"cn{idx}_{field}"
            if key in payload:
                unit[field] = payload.pop(key)

        if not unit:
            continue

        if "enabled" not in unit:
            unit["enabled"] = True

        if unit.get("image") is not None:
            unit["image"] = image_to_base64(unit.get("image"))
        if unit.get("mask") is not None:
            unit["mask"] = image_to_base64(unit.get("mask"))

        unit = deep_clean(unit)
        if unit.get("enabled"):
            controlnet_args.append(unit)

    if controlnet_args:
        payload.setdefault("alwayson_scripts", {})
        payload["alwayson_scripts"]["ControlNet"] = {"args": controlnet_args}

    payload = deep_clean(payload)
    payload = apply_conflict_rules(payload)
    payload = deep_clean(payload)
    return payload


def expected_image_count(payload):
    batch_size = int(payload.get("batch_size", 1) or 1)
    n_iter = int(payload.get("n_iter", 1) or 1)
    return max(1, batch_size) * max(1, n_iter)


def save_images_from_response(images_b64, generation_idx):
    saved = 0
    for i, b64 in enumerate(images_b64, start=1):
        image_data = b64.split(",", 1)[-1]
        file_name = f"gen_{generation_idx:04d}_{i:02d}.png"
        file_path = os.path.join(API_IMAGES_DIR, file_name)
        with open(file_path, "wb") as f:
            f.write(base64.b64decode(image_data))
        saved += 1
    return saved


def archive_outputs(tag: str):
    stamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    archive_name = os.path.join(VOLUME_DIR, f"outputs_{tag}_{stamp}.zip")
    with zipfile.ZipFile(archive_name, "w", compression=zipfile.ZIP_DEFLATED) as zf:
        for root, _, files in os.walk(OUTPUTS_DIR):
            for file in files:
                full_path = os.path.join(root, file)
                rel_path = os.path.relpath(full_path, OUTPUTS_DIR)
                zf.write(full_path, rel_path)

    for entry in os.listdir(OUTPUTS_DIR):
        p = os.path.join(OUTPUTS_DIR, entry)
        if os.path.isdir(p):
            shutil.rmtree(p)
        else:
            os.remove(p)
    os.makedirs(API_IMAGES_DIR, exist_ok=True)
    log_line(f"ARCHIVE created: {archive_name}. OUTPUTS_DIR cleaned.")


wait_for_api_ready(API_BASE)

prompts_path = first_existing_path(PROMPTS_CANDIDATES)
if not prompts_path:
    raise FileNotFoundError(
        "Файл prompts.xlsx/prompts.xlxs не найден. Ожидались пути: " + ", ".join(PROMPTS_CANDIDATES)
    )

instruction_template, generation_rows = parse_workbook(prompts_path)
log_line(f"Loaded prompts file: {prompts_path}. Rows for generation: {len(generation_rows)}")
log_line(f"Image search dirs: {INPUT_IMAGE_DIRS}")

syntax_error_dumped = False
images_since_archive = 0

total_expected_images = 0
total_saved_images = 0

for generation_idx, row_values in enumerate(generation_rows, start=1):
    try:
        payload = build_payload(row_values, instruction_template)
    except Exception as e:
        log_line(f"#{generation_idx} FAIL: payload build error: {e}")
        continue

    planned_images = expected_image_count(payload)
    total_expected_images += planned_images

    payload_dump_path = dump_payload(payload, generation_idx)
    log_line(f"#{generation_idx} request dump: {payload_dump_path}")
    if "alwayson_scripts" not in payload or not payload.get("alwayson_scripts", {}).get("ControlNet", {}).get("args"):
        log_line(f"#{generation_idx} INFO: no ControlNet image/mask attached in payload")

    try:
        response = requests.post(TXT2IMG_URL, json=payload, timeout=1800)
        if response.status_code >= 400:
            err_text = response.text[:1200]
            if not syntax_error_dumped:
                params_dump = fetch_txt2img_params_dump()
                log_line(f"#{generation_idx} FAIL syntax/validation: {response.status_code} {err_text}")
                log_line(f"#{generation_idx} FAIL payload: {payload_dump_path}")
                log_line("AVAILABLE PARAMS DUMP START")
                with open(LOG_PATH, "a", encoding="utf-8") as f:
                    f.write(params_dump + "\n")
                log_line("AVAILABLE PARAMS DUMP END")
                syntax_error_dumped = True
            else:
                log_line(f"#{generation_idx} FAIL: {response.status_code} {err_text}")
                log_line(f"#{generation_idx} FAIL payload: {payload_dump_path}")
            continue

        result = response.json()
        images = result.get("images", []) or []
        saved_now = save_images_from_response(images, generation_idx)
        total_saved_images += saved_now
        images_since_archive += saved_now

        log_line(f"#{generation_idx} OK images={saved_now} expected={planned_images}")

        if images_since_archive >= 15:
            archive_outputs(tag=f"part_{generation_idx:04d}")
            images_since_archive = 0

    except requests.exceptions.Timeout:
        log_line(f"#{generation_idx} FAIL: timeout (possible heavy generation or API freeze)")
        log_line(f"#{generation_idx} FAIL payload: {payload_dump_path}")
    except requests.exceptions.RequestException as e:
        reason = str(e)
        if "out of memory" in reason.lower() or "oom" in reason.lower():
            reason = "OOM"
        log_line(f"#{generation_idx} FAIL: {reason}")
        log_line(f"#{generation_idx} FAIL payload: {payload_dump_path}")
    except Exception as e:
        log_line(f"#{generation_idx} FAIL: unexpected error: {e}")
        log_line(f"#{generation_idx} FAIL payload: {payload_dump_path}")

if images_since_archive > 0:
    archive_outputs(tag="final")

log_line(f"Generation cycle completed. Requests={len(generation_rows)} expected_images={total_expected_images} saved_images={total_saved_images}")



[2026-02-17 20:16:22] Loaded prompts file: /kaggle/input/datasets/sokolenkotimofei/prompts/Prompts.xlsx. Rows for generation: 14
[2026-02-17 20:16:22] Image search dirs: ['/kaggle/working/gen/Images', '/kaggle/working/gen/images', '/kaggle/input/datasets/sokolenkotimofei/images']
[2026-02-17 20:16:22] #1 request dump: /kaggle/working/volume/request_dumps/request_0001.json
[2026-02-17 20:16:22] #1 INFO: no ControlNet image/mask attached in payload
[2026-02-17 20:16:23] #1 FAIL syntax/validation: 422 {"detail":[{"type":"list_type","loc":["body","hr_additional_modules"],"msg":"Input should be a valid list","input":"[]","url":"https://errors.pydantic.dev/2.8/v/list_type"}]}
[2026-02-17 20:16:23] #1 FAIL payload: /kaggle/working/volume/request_dumps/request_0001.json
[2026-02-17 20:16:23] AVAILABLE PARAMS DUMP START
[2026-02-17 20:16:23] AVAILABLE PARAMS DUMP END
[2026-02-17 20:16:23] #2 request dump: /kaggle/working/volume/request_dumps/request_0002.json
[2026-02-17 20:16:23] #2 INFO: no C

In [7]:
import requests

url = "http://127.0.0.1:17860" # замените на ваш адрес

# Получить список текущих настроек (options)
options = requests.get(f'{url}/sdapi/v1/options').json()
print(options.keys()) # Выведет все доступные имена переменных для настроек

# Получить данные о текущей модели
models = requests.get(f'{url}/sdapi/v1/sd-models').json()
print(models)

dict_keys(['samples_save', 'samples_format', 'samples_filename_pattern', 'save_images_add_number', 'save_images_replace_action', 'grid_save', 'grid_format', 'grid_extended_filename', 'grid_only_if_multiple', 'grid_prevent_empty_spots', 'grid_zip_filename_pattern', 'n_rows', 'font', 'grid_text_active_color', 'grid_text_inactive_color', 'grid_background_color', 'save_images_before_face_restoration', 'save_images_before_highres_fix', 'save_images_before_color_correction', 'save_mask', 'save_mask_composite', 'jpeg_quality', 'webp_lossless', 'export_for_4chan', 'img_downscale_threshold', 'target_side_length', 'img_max_size_mp', 'use_original_name_batch', 'use_upscaler_name_as_suffix', 'save_selected_only', 'save_write_log_csv', 'save_init_img', 'temp_dir', 'clean_temp_dir_at_start', 'save_incomplete_images', 'notification_audio', 'notification_volume', 'outdir_samples', 'outdir_txt2img_samples', 'outdir_img2img_samples', 'outdir_extras_samples', 'outdir_grids', 'outdir_txt2img_grids', 'outd